# Intro to Neural Networks Assignment

## Define the Following:
You can add image, diagrams, whatever you need to ensure that you understand the concepts below.

### Input Layer:  
this is the left-most layer of neurons of a NN, in which the data space of the network is defined.  

### Hidden Layer:  
any layer between the input layer and the output layer; receives output from an earlier layer, applies some function to that information for each neuron in the layer, and pushes the new output to the next layer in the NN.  
  
### Output Layer:  
this is the right-most layer of neurons of a NN, which uses input of nearest hidden layer to return probabilities for the outcomes of interest.
  
### Neuron:  
a container for a floating-point number that ranges between 0.0 and 1.0; columns of neurons form layers. 
  
### Weight:  
a numeric value multiplied by the respective inputs into a given layer (except for output layer?); weighting relates input data to output data.     
  
### Activation Function:  
a function that is used to reshape input data into the range of 0, 1, thus creating a unique "activation"; applied to each neuron of a given layer; ReLU (for rectified linear unit) is currently a popular activation function, $f(x)=\max(0, x)$.

### Node Map:  
a diagram of the topology of a given neural network; defining various kinds of cells (neurons), it shows how those cells combine in layers, and how those layers assemble themselves relative to one another, to form the NN.  
  
### Perceptron:  
the simplest kind of NN, it has no hidden layers; has an input layer that is weighted then passed through an activation function to return the output layer.  
  
### Bias:  
an integer used in combination with weight to optimize a given layer's explanation of patterns in data of that layer.

## Inputs -> Outputs

### Explain the flow of information through a neural network from inputs to outputs. Be sure to include: inputs, weights, bias, and activation functions. How does it all flow from beginning to end?

#### Your Answer Here

## Write your own perceptron code that can correctly classify a NAND gate. 

| x1 | x2 | y |
|----|----|---|
| 0  | 0  | 1 |
| 1  | 0  | 1 |
| 0  | 1  | 1 |
| 1  | 1  | 0 |

In [0]:
##### Your Code Here #####

## Implement your own Perceptron Class and use it to classify a binary dataset like: 
- [The Pima Indians Diabetes dataset](https://raw.githubusercontent.com/ryanleeallred/datasets/master/diabetes.csv) 
- [Titanic](https://raw.githubusercontent.com/ryanleeallred/datasets/master/titanic.csv)
- [A two-class version of the Iris dataset](https://raw.githubusercontent.com/ryanleeallred/datasets/master/Iris.csv)

You may need to search for other's implementations in order to get inspiration for your own. There are *lots* of perceptron implementations on the internet with varying levels of sophistication and complexity. Whatever your approach, make sure you understand **every** line of your implementation and what its purpose is.

In [0]:
##### Your Code Here #####

## Stretch Goals:

- Research "backpropagation" to learn how weights get updated in neural networks (tomorrow's lecture). 
- Implement a multi-layer perceptron. (for non-linearly separable classes)
- Try and implement your own backpropagation algorithm.
- What are the pros and cons of the different activation functions? How should you decide between them for the different layers of a neural network?